# Homework on Optimization

In this homework, you are asked to write a series of optimization programs with specific requirements. When I do the grading, I'll call your programs and use them to find maximizers of functions that are not in the homework. Some advice:


- The names of the programs should be the same as asked by the homework questions. Otherwise, your programs will not be found and that'll be a failure on your part.


- Make sure that your programs are robust for general use.


- Test your programs to see whether each of the options do their jobs as expected.



## Consider the following functions for optimization:

- $g(x) = -3x^3 + 9x^2 + 2x$


- $h(x,y) =  -(1.5 - x)^2 - 100(y - x^2)^2$


- $k(x,y) = - (x + 2y - 7)^2 - (2x + y -5)^2$




### [Q1] Write a Julia program to implement Newton's method of finding the maximizer of a uni-variate function. The program should be named `newton_max_uni_a` (for ease of grading) and should meet the following requirements:


- The inputs should at least contain the following items:

```julia
function newton_max_uni_a(f::Function, f′::Function, f′′::Function, x0, ϵ, maxit) 
    ...
end
```
   where `f`, `f′`, and `f′′` are the analytic equations of the objective function and its first and second derivatives, respectively; `x0` is the initial value, `ϵ` is the tolerance of convergence criterion, and `maxit` is the maximum number of iterations.


- Use the change in the solution as the basis to check convergence.


- The program should be able to check whether the found solution is a maximizer or a minimizer (hint: check the 2nd order condition). If it is a minimizer instead of a maximizer, a warning should be issued.


- The program should print at least the following information: the solution, the number of iterations, and actual ϵ. If the number of iterations equals `maxit` (meaning, the estimation may not have converged), a warning should be issued.

In [12]:
function newton_max_uni_a(f::Function, f′::Function, f′′::Function, x0, ϵ, maxit) 
    it = 1
    prev = x0 - f′(x0)/f′′(x0)
    new = 0
    while true
        new = prev - f′(prev)/f′′(prev)
        if abs(f(new) - f(prev)) <= ϵ
            break
        end
        it+=1
        if it == maxit
            break
        end
        prev = new
    end
    print("Maximized when x = ", prev,"\nϵ = ", abs(f(new) - f(prev)), "\nnumber of Iteration = ",it)
    f′′(new) > 0 && print("\nIt's a minimizer!")
    it == maxit && print("\nMaxit reached!")
    print("\n\n")
end

newton_max_uni_a (generic function with 1 method)

#### [Q2] Use the program to estimate $g(x)$. Use 0.99 and 1.01 as initial values, respectively. Do they converge to the same solution? Does the result indicate that initial values are important for Newton's methods? In light of this exercise, do you think the Newton's method is a local method or a global method?

In [13]:
g(x) = -3x^3 + 9x^2 + 2x
g′(x) = -9*(x^2) + 18* x + 2
g′′(x) = -18*x + 18
newton_max_uni_a(g,g′,g′′,0.99, 0.0001, 1000)
newton_max_uni_a(g,g′,g′′,1.01, 0.0001, 1000)

#Yes, beacuse 1 is the inflection point of g(x), Newton's method is a local method.

Maximized when x = -0.10575144969619374
ϵ = 4.382027188332138e-7
number of Iteration = 9
It's a minimizer!

Maximized when x = 2.1057514496961938
ϵ = 4.3820271855565807e-7
number of Iteration = 9



#### [Q3] Use the program to estimate $g(x)$. Use 1.0 as the initial value.

 - [Q3.1] Why this initial value does not work for the program? Do you think that the problem arises because the function has an *inflection point* at $x=1.0$?

   While x0 = 1.0, f′′(x0) = 0, so the problems arise when x0 is an inflection point.

 
 - [Q3.2] The problem indicates that the program would fail when the function is not smoothly increasing or increasing. Modify your program such that it could get out of the situation and move on. The program should be named `newton_max_uni_a2`. When you implement the rescue plan, be sure to make use of the fact that the purpose of the program is to **maximize** a function. Use the function to find the maximizer of $g(x)$.

In [21]:
function newton_max_uni_a2(f::Function, f′::Function, f′′::Function, x0, ϵ, maxit) 
    it = 1
    if f′′(x0) == 0
        if f′′(x0 + ϵ) < f′′(x0 - ϵ)
            x0 += ϵ
        else
            x0 -= ϵ
        end
    end
    
    prev = x0 - f′(x0)/f′′(x0)
    new = 0
    while true
        if f′′(prev) == 0
            if f′′(x0 + ϵ) < 0
                x0 += ϵ
            else
                x0 -= ϵ
            end
        end
        new = prev - f′(prev)/f′′(prev)
        if abs(f(new) - f(prev)) <= ϵ
            break
        end
        it+=1
        if it == maxit
            break
        end
        prev = new
    end
    print("Maximized when x = ", prev,"\nϵ = ", abs(f(new) - f(prev)), "\nnumber of Iteration = ",it)
    f′′(new) > 0 && print("\nIt's a minimizer!")
    it == maxit && print("\nMaxit reached!")
    print("\n\n")
end

newton_max_uni_a2 (generic function with 1 method)

In [22]:
g(x) = -3x^3 + 9x^2 + 2x
g′(x) = -9*(x^2) + 18* x + 2
g′′(x) = -18*x + 18
newton_max_uni_a2(g,g′,g′′,1.0, 0.0001, 1000)


Maximized when x = 2.1055572878536073
ϵ = 2.4497701645032066e-9
number of Iteration = 16



#### [Q4] Modify the above program to meet the following specifications

- Instead of requiring users to input analytic forms of the function's 1st and 2nd derivatives, use Julia's `ForwardDiff` to do it automatically. The program should be named `newton_max_uni_b`. The program should look like the following where the 1st and 2nd derivatives are calculated using `ForwardDiff` within the program. Estimate $g(x)$ using this program.

```julia
using ForwardDiff
function newton_max_uni_b(f::Function, x0, ϵ, imax)
    ....
end
```
Hint: `Forward.derivative()` is your friend. You need to find out how to use it to calculate the 2nd derivative.

In [40]:
using ForwardDiff
function newton_max_uni_b(f::Function, x0, ϵ, maxit) 
    f′(k) = ForwardDiff.derivative(f,k)
    f′′(k) = ForwardDiff.derivative(x -> ForwardDiff.derivative(f, x), k)
    it = 1
    if f′′(x0) == 0
        if f′′(x0 + ϵ) < f′′(x0 - ϵ)
            x0 += ϵ
        else
            x0 -= ϵ
        end
    end
    
    prev = x0 - f′(x0)/f′′(x0)
    new = 0
    while true
        if f′′(prev) == 0
            if f′′(x0 + ϵ) < 0
                x0 += ϵ
            else
                x0 -= ϵ
            end
        end
        new = prev - f′(prev)/f′′(prev)
        if abs(f(new) - f(prev)) <= ϵ
            break
        end
        it+=1
        if it == maxit
            break
        end
        prev = new
    end
    print("Maximized when x = ", prev,"\nϵ = ", abs(f(new) - f(prev)), "\nnumber of Iteration = ",it)
    f′′(new) > 0 && print("\nIt's a minimizer!")
    it == maxit && print("\nMaxit reached!")
    print("\n\n")
end

newton_max_uni_b (generic function with 1 method)

In [41]:
g(x) = -3x^3 + 9x^2 + 2x
newton_max_uni_b(g,1.0, 0.0001, 1000)


Maximized when x = 2.1055572878536073
ϵ = 2.4497737172168854e-9
number of Iteration = 16



### [Q5] Write a Julia program to implement Newton's method of finding the maximizers of a multi-variate function. The program should meet the following requirements:

- The program should be named `newton_max_a` (for ease of grading).


- The inputs should at least contain the following items:

```julia
function newton_max_a(f::Function, init, ϵ, maxit) 
   ...
end
```
   where `f` is the objective function, `init` is the vector of initial values, `ϵ` is the tolerance of convergence criterion, and `maxit` is the maximum number of iterations.
   

- The `f` is a function of `n` variables where `n` is larger than or equal to 1.
   

- Use `ForwardDiff` to calculate the gradient vector and the Hessian matrix.
  - Hint: `ForwardDiff.gradient()` and `ForwardDiff.hessian()`.


- Use the norm of the gradient as the basis to check convergence.
  - Hint: Google how to calculate the norm of a vector in Julia.


- The program should be able to check whether the found solution is a maximizer or a minimizer (hint: check whether the Hessian is positive definite or negative definite). If it is a minimizer instead of a maximizer, a warning should be issued.
  - Hint: Google how to check a matrix's definite using Julia.


- The program should print at least the following information: the solution, the number of iterations, and actual ϵ. If the number of iterations equals `maxit` (meaning, the estimation may not have converged), a warning should be issued.

In [42]:
using ForwardDiff, LinearAlgebra
function newton_max_a(f::Function, init, ϵ, maxit) 
    it = 1
    step = 50 * ϵ 
    prev = init
    new = init
    grad = ForwardDiff.gradient(f,prev)
    hess = ForwardDiff.hessian(f,prev)
    while true
        grad = ForwardDiff.gradient(f,prev)
        hess = ForwardDiff.hessian(f,prev)
        new = prev + step .* inv(.-hess)*grad
        gnorm = norm(grad)
        if gnorm < ϵ
            break
        end
        it += 1
        if it >= maxit 
            break
        end
        prev = new
    end
    print("Sol: ",new,"\nNumber of iterations: ",it,"\nϵ: ", norm(new))
    it == maxit && print("\nMaxit reached!")
    if LinearAlgebra.isposdef(hess[:,:])
        print("\nIt's a minimizer!")
    end
end

newton_max_a (generic function with 1 method)

#### [Q6] Use the program to estimate $h(x,y)$. Hint: Code the function as `h(x) = -(1.5 - x[1])^2 - 100*(x[2] - x[1]^2)^2` where `x` is a vector.


#### [Q7] Use the program to estimate $k(x,y)$. Hint: Code the function as `k(x) = - (x[1] + 2*x[2] - 7)^2 - (2*x[1] + x[2] -5)^2` where `x` is a vector. The estimation should converge in 1 iteration.



#### [Q8] Use the program to estimate $g(x)$. Hint: Code the function as `g(x) = -3*x[1]^3 + 9*x[1]^2 + 2*x[1]` where `x` is a vector.

In [43]:
x = [1,1]
h(x) = -(1.5 - x[1])^2 - 100*(x[2] - x[1]^2)^2
newton_max_a(h,x,0.0001,1000)

Sol: [1.4962492726654986, 2.2387535398646086]
Number of iterations: 1000
ϵ: 2.6927271117230482
Maxit reached!

In [44]:
k(x) = - (x[1] + 2*x[2] - 7)^2 - (2*x[1] + x[2] -5)^2
x = [1,3]
newton_max_a(k,x,0.0001,1000)

Sol: [1.0, 3.0]
Number of iterations: 1
ϵ: 3.1622776601683795

In [45]:
g(x) = -3*x[1]^3 + 9*x[1]^2 + 2*x[1]
x = [2]
newton_max_a(g,x,0.0001,1000)

Sol: [2.1048694513208703]
Number of iterations: 1000
ϵ: 2.1048694513208703
Maxit reached!